In [66]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

In [20]:
data = pd.read_csv('data.csv')
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [21]:
cat_columns = data.select_dtypes('object').columns
cat_columns

Index(['state', 'phone number', 'international plan', 'voice mail plan'], dtype='object')

In [22]:
data.columns

Index(['state', 'account length', 'area code', 'phone number',
       'international plan', 'voice mail plan', 'number vmail messages',
       'total day minutes', 'total day calls', 'total day charge',
       'total eve minutes', 'total eve calls', 'total eve charge',
       'total night minutes', 'total night calls', 'total night charge',
       'total intl minutes', 'total intl calls', 'total intl charge',
       'customer service calls', 'churn'],
      dtype='object')

In [54]:
churn = {True:1, False:0}
data['churn'] = data['churn'].map(churn)
y = data['churn']
X = data.iloc[:,:-1]


In [55]:
X['area code'] = X['area code'].astype(str)

In [56]:
X = X.drop('phone number', axis=1)

In [57]:
cat_cols = X.select_dtypes('object').columns
cat_cols

Index(['state', 'area code', 'international plan', 'voice mail plan'], dtype='object')

In [58]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, stratify=y, test_size=0.3,random_state=10)
lb = LabelEncoder()

In [59]:

for col in cat_cols:
    lb = LabelEncoder()
    lb.fit(xtrain[col])
    try:
        xtrain[col] = lb.transform(xtrain[col])
        xtest[col] = lb.transform(xtest[col])
    except Exception as e:
        print(col)


In [62]:
for col in xtrain.columns:
    if col in cat_cols:
        sc = StandardScaler()
        sc.fit(xtrain[col].values.reshape(-1,1))
        xtrain[col] = sc.transform(xtrain[col].values.reshape(-1,1))
        xtest[col] = sc.transform(xtest[col].values.reshape(-1,1))

,state,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
2469,1.410399,-1.335099,1.404890,-0.333889,-0.614839,-0.587746,-0.546247,2.255011,-0.546734,-0.776272,0.453969,-0.776681,-0.084799,0.450542,-0.082841,-0.694106,-1.022317,-0.696096,1.074731
1689,1.613477,-0.274695,-1.408508,-0.333889,1.626442,2.261032,-1.539364,-0.581076,-1.538997,0.349322,-0.465752,0.348462,-0.094741,-1.804738,-0.096096,0.423333,0.221849,0.425316,-0.437085
1492,0.530398,-0.931135,-1.408508,-0.333889,-0.614839,-0.587746,0.738962,1.259893,0.738697,2.006227,-0.005891,2.006080,-0.947736,-1.804738,-0.948858,-0.189456,0.221849,-0.188790,-0.437085
3049,-0.281912,1.164425,-0.001809,-0.333889,-0.614839,-0.587746,-0.885805,-0.879611,-0.886079,0.132861,2.242314,0.133157,0.048419,-1.445943,0.049713,1.937283,-0.607595,1.933883,1.074731
2865,0.259628,-0.097961,-1.408508,-0.333889,1.626442,1.749713,-1.626992,-0.680587,-1.627054,1.474916,0.658351,1.475920,-0.337318,0.348029,-0.339111,-0.585967,0.636571,-0.589295,0.318823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743,0.124243,-1.663319,-0.001809,-0.333889,-0.614839,-0.587746,1.491102,-0.680587,1.491485,1.116772,-1.129994,1.117078,-1.409029,1.168131,-1.408377,-1.595267,-1.022317,-1.590556,-0.437085
3067,1.410399,-0.981631,-0.001809,-0.333889,-0.614839,-0.587746,-1.444433,-0.133272,-1.444495,-0.548005,-0.261369,-0.547485,0.314855,1.116874,0.314820,-0.045270,-0.192873,-0.041939,-1.192994
1937,-0.755759,-0.022218,-0.001809,-0.333889,1.626442,1.822758,0.722532,0.165263,0.722589,-0.912052,2.242314,-0.910957,-2.051261,0.860593,-2.049053,0.675658,-0.192873,0.678969,-0.437085
729,-0.349604,1.265416,-0.001809,-0.333889,1.626442,1.311439,0.317253,-0.133272,0.317737,-0.502745,0.556160,-0.503498,0.390412,1.219387,0.389934,1.144262,0.221849,1.146224,-1.192994


In [68]:
lr = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=10)
scores = cross_val_score(lr, xtrain, ytrain, cv=cv, n_jobs=-1, scoring='f')


In [69]:
scores.mean()

0.284986358680472